In [1]:
import os
import json
import pandas as pd
import traceback

In [2]:
from langchain import HuggingFaceHub, LLMChain, PromptTemplate, HuggingFacePipeline
from langchain.chains import SequentialChain
from langchain.chains import SequentialChain
import PyPDF2
# from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline, AutoModelForSeq2SeqLM
import torch


/mnt/d/Academics/Generative AI by iNeuron/MCQ_generator/env/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
KEY=os.getenv("HUGGINGFACEHUB_API_TOKEN")

In [4]:
from dotenv import main

main.load_dotenv()

True

In [5]:
# model_id="nvidia/Llama-3.1-Nemotron-70B-Instruct-HF"
model_id="google/flan-t5-large"
# model_id="meta-llama/Meta-Llama-3-8B-Instruct"

In [6]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
}

In [7]:
TEMPLATE="""
Text{text}
You are a expert MCQ maker. Given the above test, it is your job to \
create a quiz of {number} multiple choice questions for {subject} students in {tone} tone.
Make sure the question are not repeated and checks all the questions to be conforming the text as well.
Make sure to format your responsse like RESPONSE_JSON below adn use it as guide. \
Ensure to make {number} MCQS
*** RESPONSE_JSON
{response_json}
"""

In [8]:
quiz_generation_prompt = PromptTemplate(
    input_variables=['text', 'number', 'subject', "tone", 'response_json'],
    template=TEMPLATE
)

In [9]:
llm=HuggingFaceHub

In [10]:
quiz_chain = LLMChain(llm=llm(repo_id=model_id, model_kwargs={'temperature':0}), prompt=quiz_generation_prompt, output_key='quiz', verbose=True)

/tmp/ipykernel_11667/3854394831.py:1: LangChainDeprecationWarning: The class `HuggingFaceHub` was deprecated in LangChain 0.0.21 and will be removed in 1.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEndpoint`.
  quiz_chain = LLMChain(llm=llm(repo_id=model_id, model_kwargs={'temperature':0}), prompt=quiz_generation_prompt, output_key='quiz', verbose=True)
/tmp/ipykernel_11667/3854394831.py:1: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  quiz_chain = LLMChain(llm=llm(repo_id=model_id, model_kwargs={'temperature':0}), prompt=quiz_generation_prompt, output_key='quiz', verbose=True)


In [11]:
TEMPLATE2="""
You are an expert english gramarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the questions and give a complete analysis of the quiz. Only use at max 50 words for complexity if the quiz is not at per with the congitive and analytical abilities of the students.\
Update the quiz questions which needs to be changed the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert  english writer of the above quiz:
"""

In [12]:
quiz_evaluation_prompt=PromptTemplate(
    input_variables=["subject", "quiz"],
    template=TEMPLATE2
)

In [13]:
review_chain = LLMChain(llm=llm(repo_id=model_id, model_kwargs={'temperature':0}), prompt=quiz_evaluation_prompt, output_key='review', verbose=True)

In [14]:
generate_evaluate_chain=SequentialChain(chains=[quiz_chain, review_chain], input_variables=["text", "number", "subject", "tone", "response_json"],
                                        output_variables=["quiz", "review"], verbose=True,)


In [15]:
file_path=r"../data.txt"

In [16]:
file_path


'../data.txt'

In [17]:
with open(file_path, 'r') as file:
    TEXT = file.read()

In [18]:
print(TEXT)

From Wikipedia, the free encyclopedia
For the journal, see Machine Learning (journal).
"Statistical learning" redirects here. For statistical learning in linguistics, see statistical learning in language acquisition.
Part of a series on
Machine learning
and data mining
Paradigms
Problems
Supervised learning
(classification • regression)
Clustering
Dimensionality reduction
Structured prediction
Anomaly detection
Artificial neural network
Reinforcement learning
Learning with humans
Model diagnostics
Mathematical foundations
Journals and conferences
Related articles
vte
Part of a series on
Artificial intelligence

Major goals
Artificial general intelligenceIntelligent agentRecursive self-improvementPlanningComputer visionGeneral game playingKnowledge reasoningNatural language processingRoboticsAI safety
Approaches
Applications
Philosophy
History
Glossary
vte
Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorit

In [19]:
# Serialize  the Python dictionary into a JSON-formatted string

json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [ ]:
import os
import json
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch

# TokenTracker class to track input and output tokens
class TokenTracker:
    def __init__(self):
        self.input_tokens = 0
        self.output_tokens = 0

    def __enter__(self):
        return self

    def __exit__(self, exc_type, exc_val, exc_tb):
        print(f"Total Input Tokens: {self.input_tokens}")
        print(f"Total Output Tokens: {self.output_tokens}")

    def add_input_tokens(self, count):
        self.input_tokens += count

    def add_output_tokens(self, count):
        self.output_tokens += count

# Load the tokenizer and model
model_id = "google/flan-t5-large"  # Example model, replace with your own
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForSeq2SeqLM.from_pretrained(model_id)

# Example usage of the function
def generate_response(text, number, subject, tone, response_json):
    # Check if any input data is missing
    if not all([text, number, subject, tone, response_json]):
        raise ValueError("All input parameters must be provided.")

    # Tokenize the input text
    input_tokens = tokenizer.encode(text, return_tensors="pt")

    # Define a fixed maximum length for the model's input
    max_length = 512  # You can adjust this value as needed for your model
    if input_tokens.size(1) > max_length:
        input_tokens = input_tokens[:, :max_length]
        print(f"Warning: Input truncated to {max_length} tokens.")

    with TokenTracker() as tracker:
        # Track the number of input tokens
        input_token_count = len(input_tokens[0])
        tracker.add_input_tokens(input_token_count)

        # Generate the output while handling memory errors
        try:
            output_tokens = model.generate(
                input_tokens, 
                max_length=min(512, input_token_count + 100),  # Ensure a reasonable max length
                num_beams=5,  # Optional: Set number of beams for beam search
                early_stopping=True
            )
        except torch.cuda.OutOfMemoryError:
            raise RuntimeError("Out of memory. Try reducing the input size or use a smaller model.")

        # Track the number of output tokens
        output_token_count = len(output_tokens[0])
        tracker.add_output_tokens(output_token_count)

        # Decode the generated output
        response_text = tokenizer.decode(output_tokens[0], skip_special_tokens=True)

    return response_text

# Example data
TEXT = TEXT
NUMBER = 5
SUBJECT = "Machine Learning"
TONE = "formal"
RESPONSE_JSON = RESPONSE_JSON

# Call the function
try:
    result = generate_response(TEXT, NUMBER, SUBJECT, TONE, RESPONSE_JSON)
    print("Generated Response:", result)
except Exception as e:
    print("An error occurred:", e)


Token indices sequence length is longer than the specified maximum sequence length for this model (13293 > 512). Running this sequence through the model will result in indexing errors


In [ ]:
# For OpenAI  
# How to setup Token Usage  Tracking  in Langchain

# with get_openai_callback() as cb:
#     response=generate_evaluate_chain(
#         {
#             "text": TEXT,
#             "number": NUMBER,
#             "subjecy": SUBJECT, 
#             "tone": TONE,
#             "respose_json": json.dumps(RESPONSE_JSON)
#         }
#     )

In [25]:
from datetime import datetime
datetime.now().strftime("%d/%m/%Y, %H:%M:%S")
datetime.now().strftime("%d_%m_%Y, %H_%M_%S")

: 

In [18]:
print( f"{datetime.now().strftime("%d/%m/%Y, %H:%M:%S")}.log")

SyntaxError: invalid syntax (212183863.py, line 1)